### TP3 - 1


1. Pretende-se construir uma implementação simplificada do algoritmo “model checking” orientado aos interpolantes seguindo a estrutura apresentada nos apontamentos onde no passo $(n,m)\,$na impossibilidade de encontrar um interpolante invariante se dá ao utilizador a possibilidade de incrementar um dos índices $n$ e $m$ à sua escolha.
    Pretende-se aplicar este algoritmo ao problema da da multiplicação de inteiros positivos em `BitVec`  (apresentado no TP2).

In [1]:
from pysmt.shortcuts import *
from pysmt.typing import INT
import itertools 

<img src="automato.png" alt="autómato" style="width: 600px;"/>

#### Função declare(t, N):

   A seguinte função cria a $t$-ésima cópia das variáveis de estado, agrupadas num dicionário que nos permite aceder às mesmas pelo nome.

In [2]:
def genState(vars, s, i, n):
    state = {}
    for v in vars:
        state[v] = Symbol(v+'!'+s+str(i), BVType(n))
    return state

#### Função init(state, prob, N):

    state - Dicionário de variáveis de estado
    a - valor associado ao 1º número a multiplicar
    b - valor associado ao 2º número a multiplicar

   A função `init` tem como objetivo devolver um predicado do Solver que testa se é um possível estado inicial do programa, através do `state`, um dicionário de variáveis.

In [3]:
def init(state, a, b, n):
    return And( 
        Equals(state['pc'], BV(0, n)),
        Equals(state['x'], BV(a, n)),
        Equals(state['y'], BV(b, n)),
        Equals(state['z'], BV(0, n))
    )

#### Função trans(curr, prox, N):

    curr - Estado das variáveis no momento atual
    prox - Estado das variáveis no momento da próxima iteração
    n - Número de bits 
    
   A função `trans` tem como objetivo devolver um predicado do Solver, através dos três estados disponíveis, que teste se é possível transitar entre os estados possíveis.

In [4]:
def error(state, n):
    return Or(
        Equals(state['pc'], BV(4, n)),
        Equals(state['pc'], BV(6, n))
    )


def trans(curr, prox, n):
    same_values = And(
        Equals(prox['x'], curr['x']),
        Equals(prox['y'], curr['y']),
        Equals(prox['z'], curr['z'])    
    )
    
    t0 = And(
        Equals(curr['pc'], BV(0, n)),
        Equals(prox['pc'], BV(1, n)),
        same_values
    )
    
    # y = 0
    t1 = And(
        Equals(curr['y'], BV(0, n)),
        Equals(curr['pc'], BV(1, n)),
        Equals(prox['pc'], BV(5, n)),
        same_values
    )
    
    # y != 0 ^ odd(y)
    t2 = And(
        NotEquals(curr['y'], BV(0, n)),
        Equals(BVURem(curr['y'], BV(2, n)), BV(1, n)),
        Equals(curr['pc'], BV(1, n)),
        Equals(prox['pc'], BV(2, n)),
        same_values
    )
    
    
    # y != 0 ^ even(y)
    t3 = And(
        NotEquals(curr['y'], BV(0, n)),
        Equals(BVURem(curr['y'], BV(2, n)), BV(0, n)),
        Equals(curr['pc'], BV(1, n)),
        Equals(prox['pc'], BV(3, n)),
        same_values
    )

    

    # transição em que o solver decide se vai para o estado de overflow ou se continua
    magia_left = And(
        Equals(prox['x'], BVLShl(curr['x'], BV(1, n))),
        Equals(prox['y'], BVLShr(curr['y'], BV(1, n))),
        Equals(prox['z'], curr['z']),
        
        Equals(curr['pc'], BV(3, n)),
        
        Or(
            And(BVUGE(prox['x'], curr['x']), Equals(prox['pc'], BV(1, n))),  # curr['x'] < prox['x']  - não há overflow
            And(BVUGT(curr['x'], prox['x']), Equals(prox['pc'], BV(4, n))),  # curr['x'] > prox['x'] - há overflow
        )
    )
    
    # transição em que o solver decide se vai para o estado de overflow ou se continua
    magia_right = And(
        Equals(prox['x'], curr['x']),
        Equals(prox['y'], BVSub(curr['y'], BV(1, n))),
        Equals(prox['z'], BVAdd(curr['z'], curr['x'])),
        
        Equals(curr['pc'], BV(2,n)),
        
        Or(
            And(BVUGE(prox['x'], curr['x']), Equals(prox['pc'], BV(1, n))),  # curr['x'] < prox['x']  - não há overflow
            And(BVUGT(curr['x'], prox['x']), Equals(prox['pc'], BV(6, n))),  # curr['x'] >= prox['x'] - há overflow
        )
    )
    
    
    # caso de paragem no overflow e no estado final
    stop_case = And(
        Equals(prox['pc'], curr['pc']),
        same_values,
        
        Or(
            And(Equals(curr['pc'], BV(4, n)), Equals(prox['pc'], BV(4, n))),
            And(Equals(curr['pc'], BV(5, n)), Equals(prox['pc'], BV(5, n))),
            And(Equals(curr['pc'], BV(6, n)), Equals(prox['pc'], BV(6, n)))
        )
    )
    
    
    return Or(t0, t1, t2, t3, stop_case, magia_left, magia_right)

#### Função gera_traco(declare, init, trans, k, n, a, b)

    declare - Variáveis de estado
    init - Condições para o estado inicial
    trans - Função transição
    k - Valor do traço
    n - Número de bits
    a - Valor a ser multiplicado
    b - Valor a ser multiplicado

A função `gera_traco` tem como objetivo imprimir o valor das variáveis à medida que vão percorrendo os estados, através das variáveis do estado, de um predicado que testa se um estado é inicial, um número positivo para gerar um possível traço de execução do programa de tamanho `k` , com `N` bits, multiplicando `a` por `b`.

In [5]:
def gera_traco(vars,init,trans, error, k, n, a, b):
    with Solver(name='z3') as s:
    
        X = [genState(vars, 'X', i, n) for i in range(k+1)]
        I = init(X[0], a, b, n)
        Tks = [trans(X[i], X[i+1], n) for i in range(k)]
     
        if s.solve([I,And(Tks)]):
            for i in range(k):
                print("Estado:",i)
                for v in X[i]:
                    print("          ",v,'=',s.get_value(X[i][v]))
                print("----------------")
        else:
            print(check)
                
gera_traco(['pc', 'x', 'y', 'z'],init,trans, error, 20, 9, 150, 2)

Estado: 0
           pc = 0_9
           x = 150_9
           y = 2_9
           z = 0_9
----------------
Estado: 1
           pc = 1_9
           x = 150_9
           y = 2_9
           z = 0_9
----------------
Estado: 2
           pc = 3_9
           x = 150_9
           y = 2_9
           z = 0_9
----------------
Estado: 3
           pc = 1_9
           x = 300_9
           y = 1_9
           z = 0_9
----------------
Estado: 4
           pc = 2_9
           x = 300_9
           y = 1_9
           z = 0_9
----------------
Estado: 5
           pc = 1_9
           x = 300_9
           y = 0_9
           z = 300_9
----------------
Estado: 6
           pc = 5_9
           x = 300_9
           y = 0_9
           z = 300_9
----------------
Estado: 7
           pc = 5_9
           x = 300_9
           y = 0_9
           z = 300_9
----------------
Estado: 8
           pc = 5_9
           x = 300_9
           y = 0_9
           z = 300_9
----------------
Estado: 9
           pc = 5_9
        

### Função invert
    - Função invert que recebe a função python que codifica a relação de transição e devolve a relação de transição inversa.

In [6]:
def invert(trans, n_bits):
    return (lambda c, p: trans(p,c, n_bits))

### O algoritmo de "model-checking"

O algoritmo de “model-checking” manipula as fórmulas $\;\mathsf{R}_n\;\equiv\; \mathsf{I}\,\land\,\mathsf{T}^n\;$ e $\;\mathsf{U}_m\equiv\; \mathsf{E}\,\land\,\mathsf{B}^m\;$ fazendo crescer os índices $\;n,m\;$. Neste exemplo, os índices $\;n,m\;$ crescem de acordo com o `input` do utilizador.

Para auxiliar na implementação deste algoritmo, começamos por definir duas funções.
A função `rename` renomeia uma fórmula (sobre um estado) de acordo com um dado estado. 
A função `same` testa se dois estados são iguais.

In [7]:
def baseName(s):
    return ''.join(list(itertools.takewhile(lambda x: x!='!', s)))

def rename(form,state):
    vs = get_free_variables(form)
    pairs = [ (x,state[baseName(x.symbol_name())]) for x in vs ]
    return form.substitute(dict(pairs))

def same(state1,state2):
    return And([Equals(state1[x], state2[x]) for x in state1])

### O algoritmo de "model-checking"


In [8]:
# Falta ver o porquê do pysmt bater mal

def model_checking(vars,init,trans,error,N, M, n_bits, a, b):
    with Solver(name="z3") as s:

        # Criar todos os estados que poderão vir a ser necessários.
        X = [genState(vars, 'X', i, n_bits) for i in range(N+1)]
        Y = [genState(vars, 'Y', i, n_bits) for i in range(M+1)]
        
    
        (n,m) = (1,1)
        command = 0
        while command != 3 and n != N and m != M:
            Tn = And([trans(X[i], X[i+1], n_bits) for i in range(n)])
            I = init(X[0], a, b, n_bits) 
            Rn = And(I, Tn)
            
            Bm = And([invert(trans, n_bits)(Y[i], Y[i+1]) for i in range(m)])
            E = error(Y[0], n_bits)
            Um = And(E, Bm)
            
            Vnm = And(Rn, same(X[n], Y[m]), Um)
            
            if s.solve([Vnm]):
                print("Unsafe")
                return
            
            else:                        # Vnm é instatisfazível
                C = binary_interpolant(And(Rn, same(X[n], Y[m])), Um)
                #C =1
                if C is None:
                    print("Interpolant None")
                    break
                C0 = rename(C, X[0])
                C1 = rename(C, X[1])
                T = trans(X[0], X[1], n_bits)
                if not s.solve([C0, T, Not(C1)]):   # C é invariante de T
                    print("Safe")
                    return
                else:
                    ### tenta gerar o majorante S
                    S = rename(C, X[n])
                    while True:
                        A = And(S, trans(X[n], Y[m], n_bits))
                        if s.solve([A,Um]):
                            print("Não é possível encontrar um majorante")
                            break
                        else:
                            Cnew = binary_interpolant(A, Um)
                            Cn = rename(Cnew, X[n])
                            if s.solve([Cn, Not(S)]):   # Se Cn -> S não é tautologia
                                S = Or(S, Cn)
                            else:             # S foi encontrado
                                print("Safe")
                                return
            
            command = int(input("1- Aumentar n\n2- Aumentar m\n3- Sair\nOpção: "))
            if command == 1:
                (n,m) = (n+1, m)
            elif command == 2:
                (n,m) = (n, m+1)
        
            print(f"N = {n}\nM = {m}\n")                
        print("unknown")                   


                        

model_checking(['pc', 'x', 'y', 'z'], init, trans, error, 20, 20, 9, 150, 2)            

Não é possível encontrar um majorante
1- Aumentar n
2- Aumentar m
3- Sair
Opção: 1
N = 2
M = 1

Não é possível encontrar um majorante
1- Aumentar n
2- Aumentar m
3- Sair
Opção: 1
N = 3
M = 1

Não é possível encontrar um majorante
1- Aumentar n
2- Aumentar m
3- Sair
Opção: 1
N = 4
M = 1

Não é possível encontrar um majorante
1- Aumentar n
2- Aumentar m
3- Sair
Opção: 2
N = 4
M = 2

Não é possível encontrar um majorante
1- Aumentar n
2- Aumentar m
3- Sair
Opção: 2
N = 4
M = 3

Safe


In [10]:
print("GG")

GG
